In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("data/season_1/features/51.csv")

In [4]:
df.dtypes

start_district_num                                                                              int64
time_slice                                                                                      int64
date                                                                                           object
order_count                                                                                   float64
succeed_count                                                                                 float64
gap                                                                                           float64
weekday                                                                                         int64
is_workday                                                                                      int64
order_30min                                                                                   float64
succeed_30min                                                                     

In [5]:
df["date"] = df["date"].apply(lambda x: pd.to_datetime(x, errors='coerce'))

In [6]:
df.dtypes

start_district_num                                                                                     int64
time_slice                                                                                             int64
date                                                                                          datetime64[ns]
order_count                                                                                          float64
succeed_count                                                                                        float64
gap                                                                                                  float64
weekday                                                                                                int64
is_workday                                                                                             int64
order_30min                                                                                          float64
succeed_30min      

In [7]:
df

,start_district_num,time_slice,date,order_count,succeed_count,gap,weekday,is_workday,order_30min,succeed_30min,gap_30min,"('order_20min', 'succeed_20min', 'gap_20min', 'order_10min', 'succeed_10min', 'gap_10min')",order_20min,succeed_20min,gap_20min,order_10min,succeed_10min,gap_10min
0,51,1,2016-01-02,348.0,191.0,157.0,5,0,347.0,252.0,95.0,0,357,203,154,300,168,132
1,51,2,2016-01-02,317.0,183.0,134.0,5,0,357.0,203.0,154.0,0,300,168,132,348,191,157
2,51,3,2016-01-02,246.0,134.0,112.0,5,0,300.0,168.0,132.0,0,348,191,157,317,183,134
3,51,4,2016-01-02,283.0,146.0,137.0,5,0,348.0,191.0,157.0,0,317,183,134,246,134,112
4,51,5,2016-01-02,289.0,155.0,134.0,5,0,317.0,183.0,134.0,0,246,134,112,283,146,137
5,51,6,2016-01-02,206.0,122.0,84.0,5,0,246.0,134.0,112.0,0,283,146,137,289,155,134
6,51,7,2016-01-02,196.0,110.0,86.0,5,0,283.0,146.0,137.0,0,289,155,134,206,122,84
7,51,8,2016-01-02,193.0,105.0,88.0,5,0,289.0,155.0,134.0,0,206,122,84,196,110,86
8,51,9,2016-01-02,141.0,91.0,50.0,5,0,206.0,122.0,84.0,0,196,110,86,193,105,88
9,51,10,2016-01-02,138.0,90.0,48.0,5,0,196.0,110.0,86.0,0,193,105,88,141,91,50


In [8]:
from sklearn.metrics.scorer import make_scorer
import math

def MAPE_scorer(y, y_pred):
    error = 0
    num = len(y)
    for i in range(0, num):
        if y[i] > 0:
            error += math.fabs(y_pred[i] - y[i]) / y[i]
    #print('error, num:', error, num)
    if num > 0:
        return error / num
    else:
        return 0

my_scorer = make_scorer(MAPE_scorer)

In [10]:
y = df['gap'][:21*66*144]
MAPE_scorer(y, [1 for i in range(0, 21*66*144)])

error, num: 53370.3598616 199584


0.26740800796433484

In [9]:
# split training, CV, test set

training = df.loc[(df.date < '2016-01-17') | (df.date == '2016-01-18')]

training_time = training.loc[df.time_slice.isin([46, 58, 70, 82, 94, 106, 118, 130, 142])]

cv = df.loc[df['date'].isin(['2016-01-17','2016-01-19','2016-01-20',
                             '2016-01-21'])]

cv_time = cv.loc[df.time_slice.isin([46, 58, 70, 82, 94, 106, 118, 130, 142])]
# only catch time slice in test set

test = df.loc[(df['date'].isin(['2016-01-23','2016-01-27','2016-01-31']) &\
         df['time_slice'].isin([46, 58, 70, 82, 94, 106, 118, 130, 142]))
      |\
       (df['date'].isin(['2016-01-25','2016-01-29']) &\
         df['time_slice'].isin([58, 70, 82, 94, 106, 118, 130, 142]))]

In [126]:

import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

In [19]:
from sklearn import linear_model
y_cv_real_all = []
y_cv_pred_all = []
y_test_all = []
for time in [46, 58, 70, 82, 94, 106, 118, 130, 142]:
    training_ts = training_time.loc[training_time.time_slice == time]
    cv_ts = cv_time.loc[cv.time_slice == time]
    test_ts = test.loc[test.time_slice == time]
    X_training = training_ts.as_matrix(columns=[
                                  'is_workday',
                                  'gap_30min',
                                  'gap_20min', 
                                  'gap_10min'])
    y_training = training_ts.as_matrix(columns=['gap']).ravel()
    X_cv = cv_ts.as_matrix(columns=[
                                  'is_workday',
                                  'gap_30min',
                                  'gap_20min', 
                                  'gap_10min'])
    y_cv = cv_ts.as_matrix(columns=['gap']).ravel()
    X_test = test_ts.as_matrix(columns=[
                                  'is_workday',
                                  'gap_30min',
                                  'gap_20min', 
                                  'gap_10min'])
    clf = linear_model.LinearRegression()
    clf.fit(X_training, y_training)
    #print(time, clf.score(X_training, y_training), clf.score(X_cv, y_cv),
    #     (MAPE_scorer(y_cv, clf.predict(X_cv))))
    #print(clf.coef_ ,clf.predict(X_test))
    MAPE_train = MAPE_scorer(y_training, modifier(clf.predict(X_training)))
    MAPE_cv = MAPE_scorer(y_cv, modifier(clf.predict(X_cv)))
    MAPE_1_train = MAPE_scorer(y_training, [1 for x in range(len(y_training))])
    MAPE_1_cv = MAPE_scorer(y_cv, [1 for x in range(len(y_cv))])
    if MAPE_train > MAPE_1_train or MAPE_cv > MAPE_1_cv:
        print('time_slice {}: train {:.2}, cv {:.2}'.format(time, 
            MAPE_train, MAPE_cv))
        print('time_slice {}: baseline {:.2}, {:.2}'.format(time, 
               MAPE_1_train ,
               MAPE_1_cv ))
    X_test = test_ts.as_matrix(columns=[
                                  'is_workday',
                                  'gap_30min',
                                  'gap_20min', 
                                  'gap_10min'])
    y_cv_pred = modifier(clf.predict(X_cv))
    y_test = modifier(clf.predict(X_test))
    
    #print(y_test)
    #print(modifier(y_test))
    print(y_cv_pred)
    print(y_cv)
    y_cv_real_all += list(y_cv)
    y_cv_pred_all += list(y_cv_pred)
    y_test_all += list(y_test)

time_slice 46: train 0.9, cv 0.77
time_slice 46: baseline 0.88, 0.97
[ 63.02787038  20.40405936  25.60042238  14.06458641]
[ 26.  41.  50.  41.]
[  5.09594079  12.09181008   8.35466455   7.58320505]
[ 3.  9.  8.  7.]
[   5.59517848   13.14947984  101.03134142   16.24605233]
[  11.   15.  205.    8.]
[  31.63520938  147.99784342   91.80054154   15.44438864]
[  17.  170.  228.   14.]
time_slice 94: train 1.4, cv 0.56
time_slice 94: baseline 0.96, 0.99
[  90.48501972  192.78460418  145.85195449   14.31216905]
[ 130.  345.  430.   84.]
[  37.71298634  837.85636824  903.90126152  469.28430259]
[   12.  1013.   910.  1109.]
[   16.26331866   274.40904432  1100.08277178    29.40364027]
[  18.  165.  551.   26.]
[ 263.34748523  690.18672834  698.50883863  571.69425501]
[ 379.  734.  882.  421.]
[  1.          33.95020791  20.57817323  87.29164605]
[ 10.  15.  44.  89.]


In [11]:
def modifier(y_pred):
    #print(y_pred)
    for i in range(len(y_pred)):
        if y_pred[i] < 1:
            y_pred[i] = 1
    #print(y_pred)
    return y_pred
#modifier(list(y_training))

In [18]:
print(MAPE_scorer(y_cv_real_all, y_cv_pred_all))
print(MAPE_scorer(y_cv_real_all, [1 for i in range(len(y_cv_real_all))]))

0.520895048586
0.953647426794


In [199]:
test = test.sort_values(by=['time_slice','date'])
test['y'] = y_test_all

In [200]:
test[['start_district_num','date','time_slice','y']]

,start_district_num,date,time_slice,y
3069,51,2016-01-23,46,1.000000
3645,51,2016-01-27,46,20.536544
4221,51,2016-01-31,46,1.000000
3081,51,2016-01-23,58,17.709742
3369,51,2016-01-25,58,30.327616
3657,51,2016-01-27,58,8.280543
3945,51,2016-01-29,58,25.864625
4233,51,2016-01-31,58,6.116048
3093,51,2016-01-23,70,9.634943
3381,51,2016-01-25,70,9.147614


In [204]:
test[['start_district_num','date','time_slice','y']].to_csv('pred51.csv')

In [208]:
X_test.shape[0]

5

In [209]:
X_test

array([[   0.,  226.,  138.,  181.],
       [   1.,   86.,   87.,   88.],
       [   1.,   73.,   79.,  105.],
       [   1.,  436.,  497.,  262.],
       [   0.,   32.,   24.,   16.]])

In [88]:
from sklearn.svm import SVR

clf = SVR(C=1.0, epsilon=0.2)
clf = clf.fit(X_training, y_training)

print(clf.score(X_training, y_training))
print(clf.score(X_cv, y_cv))
print(clf.score(X_cv_time, y_cv_time))

0.0297957532569
-0.0205079220299
-0.0923028979494


In [95]:
print(clf.predict(X_cv_time))
print(y_cv_time)

[  2.7   2.8   9.3   4.3   5.4   8.1   6.2   5.6   2.6   2.4   4.1   6.
   4.1   3.3   4.    6.2   3.5   3.2   2.7   3.6   9.2   3.    5.1   8.5
   7.7   8.2   2.6   8.8   5.7   4.    3.6   6.2  20.3   3.9   6.3   3.4]
[  3.   0.   8.   4.   4.  10.   9.   7.   4.   6.   6.   6.   3.   2.   1.
   6.   0.   1.   2.   0.  17.   1.   5.  13.   9.  11.   1.   5.   4.   2.
   2.   5.  12.   4.   7.   4.]


In [100]:

print(MAPE_scorer(y_cv, clf.predict(X_cv)))
print(MAPE_scorer(y_cv_time, clf.predict(X_cv_time)/2))

error, num: 282.053309219 576
0.489675884061
error, num: 15.7112891928 36
0.436424699801


In [104]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(training.as_matrix(columns=['time_slice']))
enc.n_values_

array([145])

In [105]:
enc.feature_indices_

array([  0, 145])

In [158]:
training.to_csv('dist51_training.csv')